In [1]:
using Distributions
using Rocket
using ReactiveMP
using BenchmarkTools

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1278
┌ Warning: Package ReactiveMP does not have LinearAlgebra in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added LinearAlgebra as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading LinearAlgebra into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function kalman_filter_graph()
    model = Model(DefaultMessageGate())
    
    x_prev = add!(model, datavar(:x_prior, Normal{Float64}))
    add_1  = add!(model, constvar(:add_1, 1.0))
    x      = add!(model, randomvar(:x))
    
    x_prev_add = add!(model, make_node(+, x_prev, add_1, x))
    
    noise = add!(model, constvar(:noise, Normal(0.0, sqrt(200.0))))
    y     = add!(model, datavar(:y, Float64))
    
    add_x_and_noise = add!(model, make_node(+, x, noise, y))
    
    activate!(model)
    
    return x_prev, x, y
end

function kalman(data)
    N = length(data)
    
    x_prev, x, y = kalman_filter_graph()

    link(x_prev, getmarginal(x))
    
    update!(x_prev, Normal(0.0, sqrt(10000.0)))
    
    marginals = Vector{Normal{Float64}}(undef, N)
    
    subscribe!(getmarginal(x) |> enumerate(), (t) -> marginals[t[1]] = getdata(t[2]))
    
    for d in data
        update!(y, d)
    end
    
    return marginals
end

kalman (generic function with 1 method)

In [3]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [4]:
@time kalman(data); # Initial compilation

  2.419334 seconds (7.08 M allocations: 380.857 MiB, 3.11% gc time)


In [5]:
@time kalman(data); # Subsequent runs

  0.000821 seconds (15.54 k allocations: 615.375 KiB)


In [6]:
@btime kalman($data) # Performance benchmark

  481.658 μs (15538 allocations: 615.38 KiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=19.995664077451256, σ=14.002800840280099)
 Normal{Float64}(μ=11.70175148263365, σ=9.950371902099892)
 Normal{Float64}(μ=5.810744539898585, σ=8.137884587711596)
 Normal{Float64}(μ=5.542674101924024, σ=7.053456158585984)
 Normal{Float64}(μ=6.442771000417222, σ=6.311944030978033)
 Normal{Float64}(μ=8.531067847995272, σ=5.763904177042351)
 Normal{Float64}(μ=11.11374546872648, σ=5.337605126836238)
 Normal{Float64}(μ=11.766067877732775, σ=4.993761694389224)
 Normal{Float64}(μ=12.927396679832668, σ=4.708816093480111)
 Normal{Float64}(μ=15.096845732555666, σ=4.467670516087703)
 Normal{Float64}(μ=16.031424658340935, σ=4.2601432284230505)
 Normal{Float64}(μ=17.939240305039593, σ=4.079085082240021)
 Normal{Float64}(μ=18.07412209449948, σ=3.919309008348103)
 ⋮
 Normal{Float64}(μ=589.5903126994336, σ=0.582706653680032)
 Normal{Float64}(μ=590.5704623368197, σ=0.5822126418123303)
 Normal{Float64}(μ=591.583970997735, σ=0.5817198842703781)
 Norma